# Assignment

In this tutorial we create a global classifier to detect pulmonary infection (pneumonia) on chest radiographs. Given the small dataset of 100 training and 100 test exams, strategies to mitigate the effect of CNN overfitting on small size data cohorts will be required. 

This assignment is part of the class **Introduction to Deep Learning for Medical Imaging** at University of California Irvine (CS190); more information can be found: https://github.com/peterchang77/dl_tutor/tree/master/cs190.

### Submission

Once complete, the following items must be submitted:

* final `*.ipynb` notebook
* final trained `*.hdf5` model file
* final compiled `*.csv` file with performance statistics

# Google Colab

The following lines of code will configure your Google Colab environment for this assignment.

### Enable GPU runtime

Use the following instructions to switch the default Colab instance into a GPU-enabled runtime:

```
Runtime > Change runtime type > Hardware accelerator > GPU
```

### Mount Google Drive

The Google Colab environment is transient and will reset after any prolonged break in activity. To retain important and/or large files between sessions, use the following lines of code to mount your personal Google drive to this Colab instance:

In [87]:
try:
    # --- Mount gdrive to /content/drive/My Drive/
    from google.colab import drive
    drive.mount('/content/drive')
    
except: pass

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Throughout this assignment we will use the following global `MOUNT_ROOT` variable to reference a location to store long-term data. If you are using a local Jupyter server and/or wish to store your data elsewhere, please update this variable now.

In [0]:
# --- Set data directory
MOUNT_ROOT = '/content/drive/My Drive'

### Select Tensorflow library version

This assignment will use the (new) Tensorflow 2.0 library. Use the following line of code to select this updated version:

In [1]:
# --- Select Tensorflow 2.0 (only in Google Colab)
% tensorflow_version 2.x
% pip install tensorflow-gpu==2.1

     |████████████████████████████████| 421.8MB 31kB/s 
     |████████████████████████████████| 3.9MB 30.2MB/s 
     |████████████████████████████████| 450kB 34.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=aea7628cd173e10e61f43df572ed3e1873aee287195cf53b8c6748a93dd5c9da
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.1.1 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
ERROR: tensorflow-probability 0.10.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorboard 2.2.1
    Uninstalling tens

# Environment

### Jarvis library

In this notebook we will Jarvis, a custom Python package to facilitate data science and deep learning for healthcare. Among other things, this library will be used for low-level data management, stratification and visualization of high-dimensional medical data.

In [2]:
# --- Install jarvis (only in Google Colab or local runtime)
% pip install jarvis-md

     |████████████████████████████████| 71kB 2.7MB/s 
     |████████████████████████████████| 276kB 9.4MB/s 
  Created wheel for pyyaml: filename=PyYAML-5.3.1-cp36-cp36m-linux_x86_64.whl size=44621 sha256=d7f26979aebe77956ee529227c72f0c62aa6e4614e3a4e7aabc5d25b522e7e0b
  Stored in directory: /root/.cache/pip/wheels/a7/c1/ea/cf5bd31012e735dc1dfea3131a2d5eae7978b251083d6247bd
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


### Imports

Use the following lines to import any additional needed libraries:

In [0]:
import numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers, metrics, regularizers
from jarvis.train import datasets, custom
from jarvis.train.client import Client
from jarvis.utils.general import overload, tools as jtools
from jarvis.utils.display import imshow

# Data

The data used in this tutorial will consist of (frontal projection) chest radiographs from the RSNA / Kaggle pneumonia challenge (https://www.kaggle.com/c/rsna-pneumonia-detection-challenge). The chest radiograph is the standard screening exam of choice to identify and trend changes in lung disease including infection (pneumonia). To simulate the problem of small dataset size, only 100 exams will be used for training (50 normal, 50 positive). A separate 100 exams will be used for independent testing.

The custom `datasets.download(...)` method can be used to download a local copy of the dataset. By default the dataset will be archived at `/data/raw/xr_pna`; as needed an alternate location may be specified using `datasets.download(name=..., path=...)`. 

In [4]:
# --- Download dataset
datasets.download(name='xr/pna-mul')

[ 2020-06-01 08:51:15 ] [====================] 100.000% : Extracting archive (0000608 / 0000608) 

{'code': '/data/raw/xr_pna', 'data': '/data/raw/xr_pna'}

# Training

To account for the small training cohort size, three separate strategies must be employed:

1. A **dual loss function** network with both classification and segmentation losses (as demonstrated in the tutorial)

2. A reduced final feature map of either (`8 x 8`) or (`4 x 4`) instead of the (`16 x 16`) used in the demonstration

3. At least one of the following (or both) regularizer techniques:

* dropout
* L2 regularization

Feel free to further optimize the network architecture to further improve model performance. Recommendings include limiting model size to reduce overfitting, early stopping and mixed L1/L2 regularization.

### Overload the `Client` object

*Hint*: Ensure to customize the `arrays['xs']['msk']` object to reflect masked values as shown in the tutorial.

In [0]:
@overload(Client)
def preprocess(self, arrays, **kwargs):
    """
    Method to create a custom msk array for class weights and/or masks
    
    """
    # --- Create msk
    msk = np.zeros(arrays['xs']['dat'].shape)

    lng = arrays['xs']['msk']
    pna = arrays['ys']['pna-seg']

    msk[lng > 0] = 1
    msk[pna > 0] = 1

    arrays['xs']['msk'] = msk
    
    return arrays

### Create `Client` object

After manually overloading the `Client` object, manually create a new client object.

In [0]:
# --- Find client yml file
yml = '{}/data/ymls/client-mul.yml'.format(jtools.get_paths('xr/pna')['code'])

# --- Manually create Client
client = Client(yml)

### Create inputs and generators

In [0]:
# --- Manually create generators
gen_train, gen_valid = client.create_generators()

# --- Create model inputs
inputs = client.get_inputs(Input)


### Define the model

*Hint*: Ensure to use both a classification and segmentation (contracting-encoding) architecture as demonstrated in the tutorial. At this point, also ensure to use either dropout or L2 regularization; other regularization techniques may also be explored to improve model performance.

In [0]:
# --- Define kwargs dictionary
kwargs = {
    'kernel_size': (1, 3, 3),
    'padding': 'same',
    'kernel_regularizer': regularizers.l2(0.01)}

# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.LeakyReLU()(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=(1, 2, 2))))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=(1, 2, 2))))

# --- Define dropout
drop = layers.Dropout(rate=0.25)

In [0]:
# --- Define contracting layers
l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
l6 = conv1(80, conv2(80, l5))
l6_conv = drop(conv1(96, conv2(96, l6)))


# --- Define expanding layers
l6_tran = tran2(80, l6_conv)
l7  = tran2(64, conv1(80, l6_tran  + l6))
l8  = tran2(48, conv1(64, l7  + l5))
l9  = tran2(32, conv1(48, l8  + l4))
l10 = tran2(16, conv1(32, l9  + l3))
l11 = tran2(8,  conv1(16, l10 + l2))
l12 = conv1(8,  conv1(8,  l11 + l1))

# --- Create classifier feature vector
c1 = layers.Reshape((-1, 1, 1, 8 * 8 * 96))(l6_conv)

# --- Create logits
logits = {}
logits['pna-seg'] = layers.Conv3D(filters=2, name='pna-seg', **kwargs)(l12)
logits['pna-cls'] = layers.Conv3D(filters=2, kernel_size=(1, 1, 1), name='pna-cls')(c1)

# --- Create model
model = Model(inputs=inputs, outputs=logits) 

### Compile the model

*Hint*: Ensure that custom loss functions are used as described in the tutorial to properly mask the segmentation loss function.

In [0]:
# --- Compile the model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={
        'pna-seg': custom.sce(inputs['msk']),
        'pna-cls': losses.SparseCategoricalCrossentropy(from_logits=True)},
    metrics={
        'pna-seg': custom.dsc(weights=inputs['msk']),
        'pna-cls': metrics.SparseCategoricalAccuracy()
        },
    experimental_run_tf_function=False)

### Train the model

Use the following cell block to train your model.

In [81]:
# --- Load data into memory for faster training
client.load_data_in_memory()

# --- Train model
model.fit(
    x=gen_train, 
    steps_per_epoch=100, 
    epochs=6,
    validation_data=gen_valid,
    validation_steps=100,
    validation_freq=2,
    use_multiprocessing=True)

[ 2020-06-01 10:39:00 ] [====================] 100.000% : Iterating | 000200    WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.
Epoch 1/6
100/100 [==============================] - 86s 858ms/step - loss: 10.2381 - pna-cls_loss: 0.1439 - pna-seg_loss: 0.1490 - pna-cls_sparse_categorical_accuracy: 0.9400 - pna-seg_dsc_1: 0.1837
Epoch 2/6
 99/100 [============================>.] - ETA: 0s - loss: 7.7180 - pna-cls_loss: 0.0025 - pna-seg_loss: 0.0872 - pna-cls_sparse_categorical_accuracy: 1.0000 - pna-seg_dsc_1: 0.3133WARNING:tensorflow:multiprocessing can interact badly with TensorFlow, causing nondeterministic deadlocks. For high performance data pipelines tf.data is recommended.
Epoch 1/6
100/100 [==============================] - 100s 1s/step - loss: 7.7104 - pna-cls_loss: 0.0025 - pna-seg_loss: 0.0870 - pna-cls_sparse_categorical_accuracy: 1.0000 - pna-seg_dsc_1: 0.315

# Evaluation

Based on the tutorial discussion, use the following cells to calculate model performance. Only model accuracy (global classification performance) needs to evaluated (Dice score does not).

### Performance

The following minimum performance metrics must be met for full credit:

* accuracy: >0.82

In [0]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

### Results

When ready, create a `*.csv` file with your compiled **validation** cohort accuracy statistics. There is no need to submit training performance accuracy.

In [83]:
accuracy = []

for x, y in test_valid:
    
    # --- Create prediction
    logits = model.predict(x)
    pred = np.argmax(logits[0], axis=-1)
    
    # --- Compare with ground truth
    accuracy.append(pred.squeeze() == y['pna-cls'].squeeze())

accuracy = np.array(accuracy)

[ 2020-06-01 10:47:59 ] [====================] 100.000% : Iterating | 000100    

In [0]:
# --- Define columns
df = pd.DataFrame(index=np.arange(accuracy.size))
df['accuracy'] = accuracy

In [85]:
print (df['accuracy'].mean())

0.88


In [0]:
import os
# --- Serialize *.csv
fname = '{}/models/disease_characterization/results.csv'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
df.to_csv(fname)

# Submission

Use the following line to save your model for submission (in Google Colab this should save your model file into your personal Google Drive):

In [0]:
# --- Serialize a model
fname = '{}/models/disease_characterization/final.hdf5'.format(MOUNT_ROOT)
os.makedirs(os.path.dirname(fname), exist_ok=True)
model.save(fname)

### Canvas

Once you have completed this assignment, download the necessary files from Google Colab and your Google Drive. You will then need to submit the following items:

* final (completed) notebook: `[UCInetID]_assignment.ipynb`
* final (results) spreadsheet: `[UCInetID]_results.csv`
* final (trained) model: `[UCInetID]_model.hdf5`

**Important**: please submit all your files prefixed with your UCInetID as listed above. Your UCInetID is the part of your UCI email address that comes before `@uci.edu`. For example, Peter Anteater has an email address of panteater@uci.edu, so his notebooke file would be submitted under the name `panteater_notebook.ipynb`, his spreadshhet would be submitted under the name `panteater_results.csv` and and his model file would be submitted under the name `panteater_model.hdf5`.